In [ ]:
from collections import deque
from constants import * # import API_KEY
from pyspark import SparkContext
import requests

# Start PySpark
sc.stop() # stop any existing contexts
sc = SparkContext.getOrCreate()
BASE_API = "https://api.themoviedb.org/3"
# Make the movie list a deque so that up to 10 recommended movies can show and older movies can be dequeued
movie_list = deque(maxlen=10)

# Dictionary of genre ids -> genre names (gotten from endpoint /genre/movie/list)
GENRES = {
  28: "Action",
  12: "Adventure",
  16: "Animation",
  35: "Comedy",
  80: "Crime",
  99: "Documentary",
  18: "Drama",
  10751: "Family",
  14: "Fantasy",
  36: "History",
  27: "Horror",
  10402: "Music",
  9648: "Mystery",
  10749: "Romance",
  878: "Science Fiction",
  10770: "TV Movie",
  53: "Thriller",
  10752: "War",
  37: "Western"
}

def update_watchlist(movies, num_movies=10):
    # Ask the user if they want to add movies to their watchlist
    print("Which movies interest you?")
    movie_choices = input("Enter the numbers in the list separated by a space (e.g. 1 2 3): ").split()
    
    # Add the id of the movie to the movie list for future GET requests
    for index in movie_choices:
        try:
            movie_index = int(index)
            
            if movie_index >= 1 and movie_index <= num_movies:
                movie_id = movies["results"][movie_index - 1]["id"]
                movie_list.append(movie_id)
                print(f"Added {movies['results'][movie_index - 1]['title']} to your watchlist")
        except ValueError:
            continue # ignore any invalid inputs

def search_by_name():
    # /search/movie
    movie_query = input("Which movie would you like to see? ")

    # Search for a movie from TMDb
    req = requests.get(f"{BASE_API}/search/movie?api_key={API_KEY}&query={movie_query}")
    movies = req.json()
    
    if req.status_code != 200:
        print(f"Error {movies['status_code']}: {movies['status_message']}")
        return
    elif movies["total_results"] == 0:
        print("No results found.")
        return
    
    # Limit to the first 10 results
    print("Top 10 results:")
    for index, result in enumerate(movies["results"][:10]):
        print(f"{index + 1}. {result['title']} ({result['release_date']}) - {result['overview']}\n")
    
    update_watchlist(movies)

def search_by_year():
    # /discover/movie?year=
    pass

def search_by_ratings():
    # /discover/movie?vote_average.gte=
    pass

def search_by_genre():
    # /discover/movie?with_genres=
    print(f"List of genres:\n{GENRES}")
    genre_id = 0
    
    while GENRES.get(genre_id) is None:
        try:
            genre_id = int(input("Please select a genre number: "))
            
            if GENRES.get(genre_id) is None:
                print("That's not a valid genre id, try again")
        except ValueError:
            print("That's not an integer, try again")
    
    # Search movies given the genre id (sort by popularity in descending order)
    req = requests.get(
        f"{BASE_API}/discover/movie?api_key={API_KEY}&with_genres={genre_id}&sort_by=popularity.desc")
    movies = req.json()
    
    if req.status_code != 200:
        print(f"Error {movies['status_code']}: {movies['status_message']}")
        return
    elif movies["total_results"] == 0:
        print("No results found.")
        return
    
    # Limit to the first 10 results
    print("Top 10 results:")
    for index, result in enumerate(movies["results"][:10]):
        print(f"{index + 1}. {result['title']} ({result['release_date']}) - {result['overview']}\n")
    
    update_watchlist(movies)

def popular_movies():
    # /movie/popular
    pass

def top_rated():
    # /movie/top_rated
    pass
        
def now_playing():
    # /movie/now_playing
    pass

def upcoming():
    # /movie/upcoming
    pass

def recommend_movies():
    # /movie/{movie_id}/recommendations
    if not movie_list:
        # The watchlist is empty, so there's nothing to recommend
        print("There's nothing to recommend. Add some movies to your watchlist.")
        return
    
    print("Based on your watch history, you might like:")
    result_limit = 10 // len(movie_list) # limit to up to 10 recommendations
    rec_movies = {"results": []} # make rec_movies resemble the movies dict to pass to update_watchlist
    rec_movies_index = 0
    
    # For each movie on the watchlist, show a handful of recommendations
    for movie_id in movie_list:
        req = requests.get(f"{BASE_API}/movie/{movie_id}/recommendations?api_key={API_KEY}")
        movies = req.json()
        
        if req.status_code != 200:
            print(f"Error {movies['status_code']}: {movies['status_message']}")
            continue
        
        rec = 0
        
        while rec < result_limit and rec < movies["total_results"]:
            result = movies["results"][rec]
            print(f"{rec_movies_index + 1}. {result['title']} ({result['release_date']}) - {result['overview']}\n")
            rec_movies["results"].append(result)
            rec_movies_index += 1
            rec += 1
    
    if rec_movies:
        update_watchlist(rec_movies, num_movies=len(rec_movies["results"]))

def recommend_by_year():
    pass

def recommend_by_genre():
    pass

def print_watchlist():
    # Print movie_list
    if not movie_list:
        print("Your watchlist is empty. Start adding movies.")
        return
    
    print("Your watchlist:")
    for movie_id in movie_list:
        req = requests.get(f"{BASE_API}/movie/{movie_id}?api_key={API_KEY}")
        movie = req.json()
        
        if req.status_code != 200:
            print(f"Error {movie['status_code']}: {movie['status_message']}")
            continue
        
        print(f"{movie['title']} ({movie['release_date']}) - {movie['overview']}\n")

option = -1
TOTAL_OPTIONS = 13
print("Welcome to Spark Movies!\n")

while option != TOTAL_OPTIONS:
    # Keep showing the menu until the user exits
    print("1. Search movie by name")
    print("2. Search by year")
    print("3. Search by ratings")
    print("4. Search by genre")
    print("5. Popular movies")
    print("6. Top rated movies")
    print("7. Movies in theaters")
    print("8. Upcoming movies")
    print("9. Recommend movies")
    print("10. Recommend by year")
    print("11. Recommend by genre")
    print("12. View watchlist")
    print("13. Exit")
    option = -1
    
    # Check for valid input
    while option < 1 or option > TOTAL_OPTIONS:
        try:
            option = int(input("Please select an option: "))
            
            if option < 1 or option > TOTAL_OPTIONS:
                print(f"Option must be from 1-{TOTAL_OPTIONS}, try again")
        except ValueError:
            print("That's not an integer, try again")

    if option == 1:
        search_by_name()
    elif option == 2:
        search_by_year()
    elif option == 3:
        search_by_ratings()
    elif option == 4:
        search_by_genre()
    elif option == 5:
        popular_movies()
    elif option == 6:
        top_rated()
    elif option == 7:
        now_playing()
    elif option == 8:
        upcoming()
    elif option == 9:
        recommend_movies()
    elif option == 10:
        recommend_by_year()
    elif option == 11:
        recommend_by_genre()
    elif option == 12:
        print_watchlist()
    else:
        # Exit the program
        print("Have a nice day!")
    
    print() # leave extra space at the end

# Stop PySpark
sc.stop()